In [22]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import logging
import pickle
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial
import gc
from importlib import reload
import pyreadstat
import disease_network_funs as fun_py
import re
import pyreadstat

In [5]:
reload(fun_py)

<module 'disease_network_funs' from '/home/hashjamm/codes/disease_network/disease_network_funs.py'>

In [4]:
sas_path = "/home/hashjamm/project_data/disease_network/sas_files/"
edge_pids_path = "/home/hashjamm/results/disease_network/edge_pids/"
matched_path = "/home/hashjamm/project_data/disease_network/sas_files/matched/"
ctable_path = "/home/hashjamm/results/disease_network/ctables/"
pids_info_path = "/home/hashjamm/results/disease_network/"

In [ ]:
# 각 node에 해당하는 성별, 나이대별, 시군구별, 

In [18]:
sgg_info = pd.read_excel(f'{etc_info_path}sgg_info.xlsx', engine="openpyxl")
sgg_info['sgg_code'] = sgg_info['sgg_code'].astype(str)

In [19]:
target_diseases = pd.read_sas(sas_path + "target_diseases" + ".sas7bdat")
target_diseases['abb_sick'] = target_diseases['abb_sick'].str.decode('utf-8')
target_diseases['COUNT'] = target_diseases['COUNT'].astype(int)
target_diseases['cum_freq'] = target_diseases['cum_freq'].astype(int)

In [112]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

node_pids_info = list()

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    chunk_node_pids_info = list()
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.node_pids_info_extractor), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            chunk_node_pids_info.append(result)
    
    node_pids_info.extend(chunk_node_pids_info)

    del chunk_node_pids_info
    
gc.collect()

100%|██████████| 4/4 [00:14<00:00,  3.66s/it]


0

In [113]:
node_pids_info = pd.DataFrame(node_pids_info).fillna(0)
node_pids_info_numcols = node_pids_info.columns.tolist()[1:]
node_pids_info[node_pids_info_numcols] =\
node_pids_info[node_pids_info_numcols].astype(int)

In [ ]:
node_pids_info = fun_py.node_shape_col_maker(node_pids_info)

In [8]:
node_pids_info

,node_code,width,height,sex_1_counts,sex_2_counts,age_0_counts,age_1_counts,age_2_counts,age_3_counts,age_4_counts,...,sex_2_ctrb_1_counts,sex_2_ctrb_2_counts,sex_2_ctrb_3_counts,sex_2_ctrb_4_counts,sex_2_ctrb_5_counts,sex_2_ctrb_6_counts,sex_2_ctrb_7_counts,sex_2_ctrb_8_counts,sex_2_ctrb_9_counts,sex_2_ctrb_10_counts
0,J03,1.000000,1.000000,41118,51040,1227,7231,11844,7787,5556,...,3165,3198,3706,4041,4719,5337,6200,6945,6932,6680
1,J20,0.999158,0.999158,33557,44137,1392,7547,8334,5408,3702,...,2885,2676,3076,3551,4110,4759,5237,6008,5889,5823
2,J06,0.998315,0.998315,30228,39585,1318,7861,7637,4603,3452,...,2585,2400,2858,3271,3662,4201,4824,5166,5258,5252
3,J02,0.997473,0.997473,27607,36329,1561,7785,7644,4575,3204,...,2218,2092,2446,2923,3262,3855,4456,4892,5164,4934
4,J00,0.996630,0.996630,27512,35232,1308,7750,6915,4203,3031,...,2446,2197,2525,2972,3344,3615,4196,4626,4698,4474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,T37,0.014322,0.167650,8,2,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1183,T54,0.038753,0.109520,5,5,0,0,1,0,0,...,1,0,0,1,0,0,1,0,2,0
1184,T75,0.014322,0.167650,8,2,0,0,2,0,1,...,0,0,0,0,1,0,1,0,0,0
1185,T94,0.076664,0.085088,3,7,0,1,0,3,0,...,3,0,2,0,1,0,1,0,0,0


In [14]:
kcd6 = pyreadstat.read_sas7bdat(f'{sas_path}/kcd6.sas7bdat', encoding='cp949')[0]
kcd6 = kcd6[kcd6['code'].str.len() == 3]
kcd6 = kcd6.drop_duplicates(subset=['code'])
kcd6.reset_index(drop=True, inplace=True)

In [15]:
kcd6.rename(columns = {'code' : 'node_code'}, inplace = True)

In [16]:
kcd6

,node_code,Korean,English
0,A00,콜레라,Cholera
1,A01,장티푸스 및 파라티푸스,Typhoid and paratyphoid fevers
2,A02,기타 살모넬라 감염,Other salmonella infections
3,A03,시겔라증,Shigellosis
4,A04,기타 세균성 장 감염,Other bacterial intestinal infections
...,...,...,...
2044,Z95,삼장 및 혈관 삽입물 및 이식편의 존재,Presence of cardiac and vascular implants and ...
2045,Z96,기타 기능성 삽입물의 존재,Persence of other functional implants
2046,Z97,기타 장치의 존재,Presence of other devices
2047,Z98,기타 수술후 상태,Other postsurgical states


In [20]:
node_pids_info = pd.merge(node_pids_info, kcd6, on='node_code', how='inner')

In [28]:
node_pids_info.to_csv(f'{pids_info_path}node_pids_info.csv', index = False, encoding="utf-8-sig")

In [125]:
edge_pids = []

for i in range(1, 11):
    with open(f'{edge_pids_path}edge_pids_{i}.pkl', 'rb') as f:
        edge_pids.append(pickle.load(f))

In [ ]:
for idx in tqdm(range(10)):
    
    year = idx + 1
    
    edge_pids_list = list(edge_pids[idx].items())

    chunk_size = len(edge_pids_list)//4  
    chunks = [edge_pids_list[i:i + chunk_size] for i in range(0, len(edge_pids_list), chunk_size)]

    edge_pids_info = list()

    for chunk_idx, chunk in enumerate(tqdm(chunks)):

        chunk_edge_pids_info = list()

        with Pool(processes=96) as pool:
            # 병렬 작업 실행
            results = \
            pool.map(partial(fun_py.edge_pids_info_extractor), chunk)

        # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
        for result in results:
            if result:
                chunk_edge_pids_info.append(result)
                
        edge_pids_info.extend(chunk_edge_pids_info)

        del chunk_edge_pids_info

    gc.collect()

    edge_pids_info = pd.DataFrame(edge_pids_info).fillna(0)
    edge_pids_info_numcols = edge_pids_info.columns.tolist()[2:]
    edge_pids_info[edge_pids_info_numcols] =\
    edge_pids_info[edge_pids_info_numcols].astype(int)

    edge_pids_info.to_csv(f'{pids_info_path}edge_pids_info_{year}.csv', index = False)

100%|██████████| 10/10 [2:28:20<00:00, 890.05s/it]


In [131]:
# edge_pids_info_1 = pd.read_csv(f'{pids_info_path}edge_pids_info_1.csv')
# edge_pids_info_2 = pd.read_csv(f'{pids_info_path}edge_pids_info_2.csv')
# edge_pids_info_3 = pd.read_csv(f'{pids_info_path}edge_pids_info_3.csv')
# edge_pids_info_4 = pd.read_csv(f'{pids_info_path}edge_pids_info_4.csv')
# edge_pids_info_5 = pd.read_csv(f'{pids_info_path}edge_pids_info_5.csv')
# edge_pids_info_6 = pd.read_csv(f'{pids_info_path}edge_pids_info_6.csv')
# edge_pids_info_7 = pd.read_csv(f'{pids_info_path}edge_pids_info_7.csv')
# edge_pids_info_8 = pd.read_csv(f'{pids_info_path}edge_pids_info_8.csv')
# edge_pids_info_9 = pd.read_csv(f'{pids_info_path}edge_pids_info_9.csv')
# edge_pids_info_10 = pd.read_csv(f'{pids_info_path}edge_pids_info_10.csv')